In [ ]:
import http.client
import json
from datetime import datetime
import time
from numba import njit
from itertools import repeat
import numpy as np
import pandas as pd

In [ ]:
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S UTC')

def get_current_time():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')

def calculate_percentage_difference(old_value, new_value):
    try:
        percentage_difference = ((new_value - old_value) / abs(old_value)) * 100
        return percentage_difference
    except ZeroDivisionError:
        return float('inf')


def get_bucket_list(current_price, bucket_range, n_buckets):
    """
        For initialization of buckets
    """
    rounded_current_price = round(current_price / 20) * 20
    asbp = rounded_current_price + (bucket_range * n_buckets) + 1
    bsbp = rounded_current_price - (bucket_range * n_buckets) - 1
    bid_buckets = [i for i in range(rounded_current_price, bsbp, -bucket_range)]
    bid_buckets = [[a, b] for a, b in zip(bid_buckets, bid_buckets[1:])]
    ask_buckets = [i for i in range(rounded_current_price, asbp, bucket_range)]
    ask_buckets = [[a, b] for a, b in zip(ask_buckets, ask_buckets[1:])]
    full = sorted(bid_buckets + ask_buckets)
    return full


def create_data_frame(dftype, current_price, bucket_range, n_buckets):
    if dftype == 'minu'
    df = pd.DataFrame(index=range(1, 1441, 1), columns=get_bucket_list(current_price, bucket_range, n_buckets))


# Add to orderbook class
def check_within_range(number_to_check : int, bucket_range, current_buckets_ranges : np.array):
    is_within = np.any((current_buckets_ranges[:, 0] <= number_to_check) & (number_to_check < current_buckets_ranges[:, 1]))
    if not is_within:
        new_ranges = []
        smallest_bucket = np.min(current_buckets_ranges)
        biggest_bucket = np.max(current_buckets_ranges)
        if smallest_bucket > number_to_check:
            while True:
                next_bucket_range = [smallest_bucket, smallest_bucket - bucket_range]
                new_ranges.append(next_bucket_range)
                smallest_bucket = smallest_bucket - bucket_range
                if smallest_bucket < number_to_check:
                    break
        if biggest_bucket < number_to_check:
            while True:
                next_bucket_range = [biggest_bucket, biggest_bucket + bucket_range]
                new_ranges.append(next_bucket_range)
                biggest_bucket = biggest_bucket + bucket_range
                if biggest_bucket > number_to_check:
                    break
        return new_ranges
    else:
      return None

# Update dataframe
def update_dataframe_buckets(df, bucket_ranges):
    b_names = ["_".join(bucket_ranges)]
    for name in b_names:
        df[name] = np.nan
    df.dropna(axis=1, how='all')

def get_binance_books(limit=5000):
    """
      If there are no 5000 book records, we will try with lesser amount of records
    """
    try:
        conn = http.client.HTTPSConnection("api.binance.com")
        conn.request("GET", f"/api/v3/depth?symbol=BTCUSDT&limit={limit}")
        b = json.loads(conn.getresponse().read().decode("utf-8"))
    except:
        limit = limit-100
        get_binance_books(limit)



In [ ]:
bucket_range = 20
current_price = 40000


df = pd.DataFrame(index=range(1, 1441, 1), columns=get_bucket_list(100, "asks", True))


print("elapsed: ", time.time() - t)

#### Current buckets
current_buckets_ranges = np.array([[int(part) for part in x.split('_')] for x in df.columns.tolist()])

number_to_check = 43000
t = time.time()

a = check_within_range(number_to_check, bucket_range, current_buckets_ranges)
print(a)



print(np.min(current_buckets_ranges))
print(np.max(current_buckets_ranges))

print("elapsed: ", time.time() - t)

def update_dataframe_buckets(df, bucket_ranges):
    b_names = ["_".join(bucket_ranges)]
    for name in b_names:
        df[name] = np.nan
    df.dropna(axis=1, how='all')
    



In [ ]:
class OrderBook:

    def __init__(self, exchange, insType, symbol, bucket_range, n_buckets, is_test=True):

        self.exchange = exchange
        self.insType = insType
        self.symbol = symbol
        self.bids = get_bucket_list(n_buckets, "bids", True)
        self.asks = get_bucket_list(n_buckets, "asks", True)
        self.bucket_range = bucket_range
        # Fetch Initial OrderBook
        # if "binance" in exchange and insType == 'spot' and symbol == 'BTCUSDT':
        #     if is_test == True:
        #         b = json.load(open('/content/bbooks.json'))
        #         for 
        #     if is_test == False:
        #         b = get_binance_books()


    def check_within_range(self, number_to_check : int, current_buckets_ranges : np.array):

        return np.any((current_buckets_ranges[:, 0] <= number_to_check) & (number_to_check < current_buckets_ranges[:, 1]))





    def add_order(self, order):
        side = order.side
        price = order.price
        quantity = order.quantity

        if side == "buy":
            self.add_bid(price, quantity)
        elif side == "sell":
            self.add_ask(price, quantity)
        else:
            raise ValueError(f"Invalid side: {side}")

    def add_bid(self, price, quantity):
        if price in self.bids:
            self.bids[price].quantity += quantity
        else:
            self.bids[price] = Order(price, quantity, "buy")

        # Sort bids by price in descending order
        self.bids = dict(sorted(self.bids.items(), key=lambda item: item[0], reverse=True))

    def add_ask(self, price, quantity):
        if price in self.asks:
            self.asks[price].quantity += quantity
        else:
            self.asks[price] = Order(price, quantity, "sell")

        # Sort asks by price in ascending order
        self.asks = dict(sorted(self.asks.items()))